In [33]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from pystacknet.pystacknet import StackNetRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP

import types

In [2]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [3]:
df_train = pd.read_pickle('feats/df_train.pkl')
df_test = pd.read_pickle('feats/df_test.pkl')

In [4]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [52]:
def stacknet_params(models):
    model_names = []
    for lv_idx,models_current_lv  in enumerate(models):
        model_names.append([(model.__class__.__name__,model.get_params())   for model in models_current_lv])
    return model_names

In [70]:
def process(df_train, df_test, columns, models, trial , remark='', restacking=False):
    
    X_train,y_train = df_train[columns].values, df_train['y'].values
    X_test = df_test[columns].values
    scaler = StandardScaler()
    scaler.fit(np.concatenate([X_train, X_test], axis=0))
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    stacknet = StackNetRegressor(models, metric="mae", folds=8, restacking=restacking,use_retraining=True, random_state=12345, n_jobs=4, verbose=1)
    stacknet.fit(X_train,y_train)
    y_train_pred = stacknet.predict(X_train)
    y_test_pred = stacknet.predict(X_test)
    
    trial.append({'nColumns':len(columns), 'columns':columns, 'remark':remark, 'models':stacknet_params(models), 'y_train_pred':y_train_pred, 'y_test_pred':y_test_pred, 'train_mae':mean_absolute_error(y_train, y_train_pred)})
    return

In [5]:
lgbm_param = {'columns': ['spkt_welch_density__coeff_3',
  'abs_q25_5',
  'q25_roll_std_100',
  'abs_q75_6',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'abs_q01_4',
  'iqr_6',
  'q05_roll_std_100',
  'q05_roll_std_1000',
  'median__roll_std',
  'abs_q01_5',
  "number_peaks{'n': 10}",
  'FFT_Mag_75q0',
  "value_count{'value': 1}",
  'q01_roll_std_100',
  'abs_q95_2',
  'abs_q95_6',
  'MA_1000MA_std_mean_7',
  'q05_roll_std_10',
  'q01_roll_std_1000',
  'abs_max_roll_mean_1000',
  'abs_q75_2',
  'abs_q05_6',
  '5000std_quantile25',
  "number_crossing_m{'m': 1}",
  "autocorrelation{'lag': 5}",
  'q75_roll_std_10',
  'q05_2',
  '5000smoothness_quantile05'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'lgb.LGBMRegressor',
  'init': {'learning_rate': 0.17076106120259138,
   'feature_fraction': 0.6842101917408698,
   'bagging_fraction': 0.8986268312800509,
   'min_data_in_leaf': 243,
   'lambda_l1': 4.612300279009062,
   'lambda_l2': 97.21686371760525,
   'max_bin': 28,
   'num_leaves': 11,
   'random_state': 6805,
   'n_jobs': 32},
  'fit': {'eval_metric': 'mae',
   'verbose': False,
   'early_stopping_rounds': 200}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance': False,
  'permutation_random_state': 1}}


In [6]:
xgbm_param = {'columns': ['spkt_welch_density__coeff_3',
  'abs_q25_5',
  'q25_roll_std_100',
  'abs_q75_6',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'abs_q01_4',
  'iqr_6',
  'q05_roll_std_100',
  'q05_roll_std_1000',
  'median__roll_std',
  'abs_q01_5',
  "number_peaks{'n': 10}",
  'FFT_Mag_75q0',
  "value_count{'value': 1}",
  'q01_roll_std_100',
  'abs_q95_2',
  'abs_q95_6',
  'MA_1000MA_std_mean_7',
  'q05_roll_std_10',
  'q01_roll_std_1000',
  'abs_max_roll_mean_1000',
  'abs_q75_2',
  'abs_q05_6',
  '5000std_quantile25',
  "number_crossing_m{'m': 1}",
  "autocorrelation{'lag': 5}",
  'q75_roll_std_10',
  'q05_2',
  '5000smoothness_quantile05'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'xgb.XGBRegressor',
  'init': {'max_depth': 3,
   'max_bin': 38,
   'eta': 0.27801915385245873,
   'colsample_bytree': 0.9416983653127328,
   'min_child_weight': 238,
   'n_estimators': 165,
   'subsample': 0.7471829960670435,
   'reg_lambda': 0.6813060508093833,
   'reg_alpha': 0.36085980027529035,
   'n_jobs': 16},
  'fit': {'eval_metric': 'mae',
   'verbose': False,
   'early_stopping_rounds': 200}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance': False,
  'permutation_random_state': 1}}

In [7]:
catboost_param = {'columns': ['spkt_welch_density__coeff_3',
  'abs_q25_5',
  'q25_roll_std_100',
  'abs_q75_6',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'abs_q01_4',
  'iqr_6',
  'q05_roll_std_100',
  'q05_roll_std_1000',
  'median__roll_std',
  'abs_q01_5',
  "number_peaks{'n': 10}",
  'FFT_Mag_75q0',
  "value_count{'value': 1}",
  'q01_roll_std_100',
  'abs_q95_2',
  'abs_q95_6',
  'MA_1000MA_std_mean_7',
  'q05_roll_std_10',
  'q01_roll_std_1000',
  'abs_max_roll_mean_1000',
  'abs_q75_2',
  'abs_q05_6',
  '5000std_quantile25',
  "number_crossing_m{'m': 1}",
  "autocorrelation{'lag': 5}",
  'q75_roll_std_10',
  'q05_2',
  '5000smoothness_quantile05'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 589,
   'depth': 6,
   'learning_rate': 0.05293979792364842,
   'l2_leaf_reg': 78.065140245968,
   'bagging_temperature': 0.9302786271852079,
   'random_strength': 0.4247048326178351,
   'random_state': 651},
  'fit': {'verbose': False, 'early_stopping_rounds': 200}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance': False,
  'permutation_random_state': 1}}

In [8]:
randomforest_param = {'columns': ['q25_roll_std_100',
  'abs_q01_4',
  'q05_roll_std_1000',
  'abs_q25_5',
  'spkt_welch_density__coeff_3',
  'q05_roll_std_100',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'median__roll_std',
  'min__roll_std',
  "number_peaks{'n': 10}",
  'abs_q75_6',
  'q01_roll_mean_1000',
  'abs_q75_7',
  'spkt_welch_densitycoeff_2',
  'min_roll_std_100',
  'max_to_min',
  'max_to_min_6',
  'iqr_6',
  'abs_max_8',
  'abs_max_roll_mean_100',
  'abs_max_roll_mean_1000',
  'min__roll_mean',
  'abs_max_7',
  'max_to_min_1',
  '5000rms_quantile25',
  'abs_max_6',
  'q99_roll_mean_1000',
  'max_to_min_5',
  'max_9',
  'abs_max_1',
  'skew_1',
  'mean_change_rate',
  "autocorrelation{'lag': 5}",
  'q95_roll_mean_1000',
  'abs_max_5',
  "number_crossing_m{'m': 1}",
  'max_to_min_diff_5',
  "value_count{'value': 1}",
  'abs_q99_7',
  'min_9',
  'abs_q75_2',
  'med_8',
  'abs_max_4',
  'abs_q05_2',
  "change_quantiles{'ql': 0.2, 'qh': 0.4, 'isabs': True, 'f_agg': 'mean'}",
  '5000form_factor_quantile75',
  'abs_q25_9',
  'agg_autocorrelationf_agg_"mean"__maxlag_40',
  "binned_entropy{'max_bins': 10}",
  'abs_min_2',
  'spkt_welch_density__coeff_30',
  'abs_min_5',
  'q05_2',
  'abs_q95_7',
  'kurt_7',
  'q95_9',
  'fft_coefficientcoeff_80__attr_"imag"',
  'fft_coefficientcoeff_32__attr_"imag"',
  'fft_coefficientcoeff_24__attr_"real"',
  'ave10_6',
  'abs_q01_6',
  'iqr_8',
  "change_quantiles{'ql': 0.2, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}",
  'med_5',
  'partial_autocorrelationlag_1',
  'ave10_2',
  'spkt_welch_density__coeff_42',
  'peak_to_average_power_ratio__roll_mean',
  'abs_min_7',
  'abs_q25_7',
  '5000skewness_max_',
  'spkt_welch_density__coeff_115',
  '5000skewness_mean_',
  'fft_coefficientcoeff_8__attr_"imag"',
  'abs_q95_2',
  'ave10_7',
  'abs_min_3',
  "change_quantiles{'ql': 0.6, 'qh': 0.8, 'isabs': True, 'f_agg': 'mean'}",
  'abs_min_6',
  'fft_coefficientcoeff_20__attr_"abs"'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'RandomForestRegressor',
  'init': {'n_estimators': 747,
   'max_depth': 9,
   'max_features': 0.6639543350506906,
   'min_samples_leaf': 0.10968001196429095,
   'random_state': 2519},
  'fit': {}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance': False,
  'permutation_random_state': 1}}

In [9]:
extratree_param = {'columns': ['q25_roll_std_100',
  'abs_q25_5',
  'abs_q01_4',
  'iqr_6',
  'abs_q75_6',
  'mean_change_rate',
  'abs_q25_7',
  'abs_q75_7',
  'q05_2',
  'q01_roll_std_10',
  "change_quantiles{'ql': 0.4, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'q25_1',
  'spkt_welch_density__coeff_29',
  '3th_peak_freq',
  'spkt_welch_densitycoeff_2',
  'abs_q95_2',
  'abs_q25_9',
  'abs_q25_1',
  'min__roll_std',
  'spkt_welch_density__coeff_30',
  'spkt_welch_density__coeff_28',
  'iqr_8',
  'abs_q75_1',
  'abs_q95_8',
  'abs_max_8',
  'kurt_7',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  'fft_coefficientcoeff_26__attr_"abs"',
  'abs_max_7',
  'fft_coefficientcoeff_21__attr_"abs"',
  'q95_9',
  'spkt_welch_density__coeff_3',
  'abs_q01_7',
  'abs_q99_8',
  'abs_q99_7',
  'fft_coefficientcoeff_32__attr_"imag"',
  'peak_to_average_power_ratio__roll_mean',
  'Hilbert_mean_1',
  'iqr_9',
  'fft_coefficientcoeff_20__attr_"abs"',
  'min_roll_std_100',
  'FFT_Mag_75q0',
  'fft_coefficientcoeff_16__attr_"imag"',
  "change_quantiles{'ql': 0.0, 'qh': 0.8, 'isabs': False, 'f_agg': 'mean'}",
  'partial_autocorrelationlag_5',
  'med_6',
  'abs_q25_8',
  'spkt_welch_density__coeff_63',
  'med_8',
  'med_4',
  'spkt_welch_density__coeff_65',
  "autocorrelation{'lag': 5}",
  'abs_q05_9',
  'fft_coefficientcoeff_24__attr_"imag"',
  'med_2',
  'spkt_welch_density__coeff_38',
  'fft_coefficientcoeff_50__attr_"abs"',
  'spkt_welch_density__coeff_50',
  '5000skewness_mean_',
  'ave10_7',
  'spkt_welch_density__coeff_115',
  'spkt_welch_density__coeff_117',
  'spkt_welch_density__coeff_79',
  "change_quantiles{'ql': 0.4, 'qh': 0.6, 'isabs': False, 'f_agg': 'mean'}",
  'spkt_welch_density__coeff_41',
  'partial_autocorrelationlag_1',
  'spkt_welch_density__coeff_113',
  'fft_coefficientcoeff_8__attr_"imag"',
  'spkt_welch_density__coeff_42',
  'spkt_welch_density__coeff_84'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'ExtraTreesRegressor',
  'init': {'n_estimators': 449,
   'max_depth': 13,
   'max_features': 0.9606948036865893,
   'min_samples_leaf': 0.10365931931330866,
   'random_state': 1425},
  'fit': {}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance': False,
  'permutation_random_state': 1}}

In [10]:
gbm_param = {'columns': ['q25_roll_std_100',
  'spkt_welch_density__coeff_3',
  'abs_q01_4',
  'abs_q25_5',
  'q05_roll_std_1000',
  'median__roll_std',
  'iqr_6',
  'spkt_welch_densitycoeff_2',
  'q05_roll_std_100',
  'q05_2',
  'abs_q75_7',
  "change_quantiles{'ql': 0.2, 'qh': 0.8, 'isabs': False, 'f_agg': 'var'}",
  "value_count{'value': -1}",
  'abs_q95_2',
  'MA_1000MA_std_mean_7',
  '5000smoothness_std_',
  '5000quantile25peak_to_average_power_ratio_',
  'FFT_Mag_25q0',
  '5000skewness_max_',
  'q05_5',
  'max_to_min_5',
  "autocorrelation{'lag': 5}",
  'abs_q05_2',
  'min__roll_std',
  '5000peak_peak_amp_max_',
  'max_to_min_diff_5',
  '5000form_factor_quantile75',
  'iqr',
  'kurt_7',
  'spkt_welch_density__coeff_4',
  "number_peaks{'n': 3}",
  'spkt_welch_density__coeff_42',
  'fft_coefficientcoeff_20__attr_"abs"',
  '5000quantile05median_',
  'abs_max_2',
  "number_peaks{'n': 1}",
  'abs_max_8',
  "number_peaks{'n': 5}",
  'spkt_welch_density__coeff_17',
  'abs_max_3',
  '4th_peak_psd',
  "autocorrelation{'lag': 4}",
  '5000min_quantile05',
  '5000kurtosis_mean_',
  'min_9',
  'ave10_6',
  'spkt_welch_density__coeff_57',
  'abs_max_7',
  'agg_autocorrelationf_agg_"mean"__maxlag_40',
  'spkt_welch_density__coeff_31',
  'kurt_3',
  '5000skewness_mean_',
  '5000kurtosis_quantile75',
  'abs_q95_6',
  'spkt_welch_densitycoeff_8',
  'abs_min_5',
  'med_4',
  'abs_q01_7',
  'spkt_welch_density__coeff_64',
  'abs_min_3',
  'fft_coefficientcoeff_19__attr_"abs"',
  'fft_coefficientcoeff_62__attr_"abs"',
  'spkt_welch_density__coeff_99',
  'partial_autocorrelationlag_1',
  'fft_coefficientcoeff_56__attr_"angle"',
  'fft_coefficientcoeff_6__attr_"abs"',
  'iqr_8',
  'abs_q05_1',
  'abs_max_6',
  'fft_coefficientcoeff_36__attr_"abs"'],
 'kfold': {'n_splits': 8,
  'random_state': 1985,
  'shuffle': True,
  'type': 'stratified'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'GradientBoostingRegressor',
  'init': {'max_depth': 3,
   'learning_rate': 0.04018024140881379,
   'n_estimators': 253,
   'subsample': 0.6846361552509973,
   'alpha': 0.9990262087522855,
   'random_state': 2784},
  'fit': {}},
 'feature_importance': {'is_output': False,
  'permutation_feature_importance': False,
  'permutation_random_state': 1}}

In [71]:
models=[ 
######## First level ########
    [
        # RandomForestRegressor (**randomforest_param['algorithm']['init']),
        #  ExtraTreesRegressor (**extratree_param['algorithm']['init']),
        #  GradientBoostingRegressor(**gbm_param['algorithm']['init']),
         cb.CatBoostRegressor(**catboost_param['algorithm']['init']),
         lgb.LGBMRegressor(**lgbm_param['algorithm']['init']),
         xgb.XGBRegressor(**xgbm_param['algorithm']['init']),
     ],
######## Second level ########
    [
        cb.CatBoostRegressor(**catboost_param['algorithm']['init']),
    ]
]

In [72]:
# mytrial = []

In [73]:
columns = gbm_param['columns']
process(df_train, df_test, columns, models, mytrial , remark='try gbm selected columns')

====================== Start of Level 0 ======================
Input Dimensionality 70 at Level 0 
3 models included in Level 0 
0:	learn: 6.4478137	total: 102ms	remaining: 59.9s
1:	learn: 6.1726546	total: 181ms	remaining: 53.2s
2:	learn: 5.9167665	total: 264ms	remaining: 51.7s
3:	learn: 5.6735442	total: 351ms	remaining: 51.3s
4:	learn: 5.4480158	total: 384ms	remaining: 44.9s
5:	learn: 5.2352094	total: 452ms	remaining: 43.9s
6:	learn: 5.0361522	total: 522ms	remaining: 43.4s
7:	learn: 4.8497146	total: 590ms	remaining: 42.9s
8:	learn: 4.6755448	total: 662ms	remaining: 42.7s
9:	learn: 4.5126475	total: 728ms	remaining: 42.2s
10:	learn: 4.3620768	total: 795ms	remaining: 41.7s
11:	learn: 4.2205873	total: 861ms	remaining: 41.4s
12:	learn: 4.0893051	total: 931ms	remaining: 41.2s
13:	learn: 3.9678432	total: 1s	remaining: 41.1s
14:	learn: 3.8561268	total: 1.07s	remaining: 40.9s
15:	learn: 3.7525449	total: 1.14s	remaining: 40.8s
16:	learn: 3.6569121	total: 1.21s	remaining: 40.6s
17:	learn: 3.5680

159:	learn: 2.4308135	total: 11.7s	remaining: 31.4s
160:	learn: 2.4305390	total: 11.8s	remaining: 31.3s
161:	learn: 2.4296688	total: 11.9s	remaining: 31.3s
162:	learn: 2.4291746	total: 11.9s	remaining: 31.2s
163:	learn: 2.4284500	total: 12s	remaining: 31.1s
164:	learn: 2.4279475	total: 12.1s	remaining: 31s
165:	learn: 2.4268136	total: 12.1s	remaining: 30.9s
166:	learn: 2.4255743	total: 12.2s	remaining: 30.9s
167:	learn: 2.4251151	total: 12.3s	remaining: 30.8s
168:	learn: 2.4246216	total: 12.4s	remaining: 30.7s
169:	learn: 2.4234644	total: 12.4s	remaining: 30.6s
170:	learn: 2.4226846	total: 12.5s	remaining: 30.5s
171:	learn: 2.4222951	total: 12.6s	remaining: 30.4s
172:	learn: 2.4219059	total: 12.6s	remaining: 30.4s
173:	learn: 2.4208733	total: 12.7s	remaining: 30.3s
174:	learn: 2.4198628	total: 12.8s	remaining: 30.2s
175:	learn: 2.4190309	total: 12.8s	remaining: 30.1s
176:	learn: 2.4180077	total: 12.9s	remaining: 30.1s
177:	learn: 2.4170300	total: 13s	remaining: 30s
178:	learn: 2.416503

318:	learn: 2.3211135	total: 23.2s	remaining: 19.6s
319:	learn: 2.3204061	total: 23.2s	remaining: 19.5s
320:	learn: 2.3200562	total: 23.3s	remaining: 19.5s
321:	learn: 2.3196327	total: 23.4s	remaining: 19.4s
322:	learn: 2.3190130	total: 23.5s	remaining: 19.3s
323:	learn: 2.3179991	total: 23.5s	remaining: 19.2s
324:	learn: 2.3174625	total: 23.6s	remaining: 19.2s
325:	learn: 2.3172062	total: 23.7s	remaining: 19.1s
326:	learn: 2.3165511	total: 23.7s	remaining: 19s
327:	learn: 2.3158133	total: 23.8s	remaining: 18.9s
328:	learn: 2.3150045	total: 23.9s	remaining: 18.9s
329:	learn: 2.3143857	total: 24s	remaining: 18.8s
330:	learn: 2.3139463	total: 24s	remaining: 18.7s
331:	learn: 2.3134070	total: 24.1s	remaining: 18.7s
332:	learn: 2.3129057	total: 24.2s	remaining: 18.6s
333:	learn: 2.3126275	total: 24.2s	remaining: 18.5s
334:	learn: 2.3119133	total: 24.3s	remaining: 18.4s
335:	learn: 2.3115574	total: 24.4s	remaining: 18.4s
336:	learn: 2.3109515	total: 24.4s	remaining: 18.3s
337:	learn: 2.3106

477:	learn: 2.2397690	total: 32.3s	remaining: 7.5s
478:	learn: 2.2393332	total: 32.3s	remaining: 7.42s
479:	learn: 2.2387719	total: 32.4s	remaining: 7.36s
480:	learn: 2.2384436	total: 32.4s	remaining: 7.28s
481:	learn: 2.2378739	total: 32.5s	remaining: 7.21s
482:	learn: 2.2376291	total: 32.5s	remaining: 7.14s
483:	learn: 2.2374419	total: 32.6s	remaining: 7.07s
484:	learn: 2.2367808	total: 32.7s	remaining: 7s
485:	learn: 2.2362540	total: 32.7s	remaining: 6.93s
486:	learn: 2.2358690	total: 32.8s	remaining: 6.86s
487:	learn: 2.2354148	total: 32.8s	remaining: 6.79s
488:	learn: 2.2352982	total: 32.9s	remaining: 6.72s
489:	learn: 2.2350576	total: 32.9s	remaining: 6.65s
490:	learn: 2.2347264	total: 33s	remaining: 6.58s
491:	learn: 2.2343024	total: 33s	remaining: 6.51s
492:	learn: 2.2336555	total: 33.1s	remaining: 6.44s
493:	learn: 2.2333534	total: 33.1s	remaining: 6.37s
494:	learn: 2.2326805	total: 33.2s	remaining: 6.3s
495:	learn: 2.2324334	total: 33.2s	remaining: 6.23s
496:	learn: 2.2319718

45:	learn: 2.6483922	total: 3.53s	remaining: 41.7s
46:	learn: 2.6406497	total: 3.61s	remaining: 41.6s
47:	learn: 2.6327496	total: 3.69s	remaining: 41.6s
48:	learn: 2.6257251	total: 3.77s	remaining: 41.6s
49:	learn: 2.6184049	total: 3.84s	remaining: 41.4s
50:	learn: 2.6119495	total: 3.92s	remaining: 41.3s
51:	learn: 2.6046328	total: 3.99s	remaining: 41.2s
52:	learn: 2.5995165	total: 4.06s	remaining: 41.1s
53:	learn: 2.5943989	total: 4.14s	remaining: 41s
54:	learn: 2.5892547	total: 4.21s	remaining: 40.9s
55:	learn: 2.5842950	total: 4.29s	remaining: 40.9s
56:	learn: 2.5801582	total: 4.36s	remaining: 40.7s
57:	learn: 2.5751925	total: 4.44s	remaining: 40.6s
58:	learn: 2.5706810	total: 4.51s	remaining: 40.6s
59:	learn: 2.5655384	total: 4.6s	remaining: 40.6s
60:	learn: 2.5617743	total: 4.69s	remaining: 40.6s
61:	learn: 2.5583846	total: 4.77s	remaining: 40.6s
62:	learn: 2.5554880	total: 4.84s	remaining: 40.4s
63:	learn: 2.5518301	total: 4.92s	remaining: 40.3s
64:	learn: 2.5491640	total: 4.99s	

...

473:	learn: 2.2647241	total: 3.54s	remaining: 858ms
474:	learn: 2.2647155	total: 3.55s	remaining: 853ms
475:	learn: 2.2646824	total: 3.56s	remaining: 846ms
476:	learn: 2.2646821	total: 3.57s	remaining: 838ms
477:	learn: 2.2645615	total: 3.58s	remaining: 830ms
478:	learn: 2.2645602	total: 3.58s	remaining: 822ms
479:	learn: 2.2645591	total: 3.59s	remaining: 815ms
480:	learn: 2.2645590	total: 3.59s	remaining: 807ms
481:	learn: 2.2645569	total: 3.6s	remaining: 799ms
482:	learn: 2.2645548	total: 3.61s	remaining: 792ms
483:	learn: 2.2644980	total: 3.61s	remaining: 784ms
484:	learn: 2.2644965	total: 3.62s	remaining: 776ms
485:	learn: 2.2644954	total: 3.63s	remaining: 768ms
486:	learn: 2.2644850	total: 3.63s	remaining: 761ms
487:	learn: 2.2644483	total: 3.64s	remaining: 753ms
488:	learn: 2.2643830	total: 3.65s	remaining: 746ms
489:	learn: 2.2643734	total: 3.65s	remaining: 738ms
490:	learn: 2.2643730	total: 3.66s	remaining: 730ms
491:	learn: 2.2642300	total: 3.67s	remaining: 723ms
492:	learn: 2

59:	learn: 2.3140632	total: 479ms	remaining: 4.22s
60:	learn: 2.3121380	total: 490ms	remaining: 4.24s
61:	learn: 2.3103361	total: 498ms	remaining: 4.23s
62:	learn: 2.3086540	total: 506ms	remaining: 4.22s
63:	learn: 2.3071212	total: 513ms	remaining: 4.21s
64:	learn: 2.3059041	total: 520ms	remaining: 4.19s
65:	learn: 2.3046845	total: 527ms	remaining: 4.18s
66:	learn: 2.3035280	total: 534ms	remaining: 4.16s
67:	learn: 2.3023741	total: 541ms	remaining: 4.15s
68:	learn: 2.3015365	total: 549ms	remaining: 4.14s
69:	learn: 2.3008158	total: 556ms	remaining: 4.12s
70:	learn: 2.3000637	total: 563ms	remaining: 4.11s
71:	learn: 2.2993464	total: 571ms	remaining: 4.1s
72:	learn: 2.2986993	total: 578ms	remaining: 4.09s
73:	learn: 2.2979325	total: 586ms	remaining: 4.08s
74:	learn: 2.2971123	total: 593ms	remaining: 4.06s
75:	learn: 2.2962765	total: 600ms	remaining: 4.05s
76:	learn: 2.2957416	total: 607ms	remaining: 4.04s
77:	learn: 2.2953184	total: 615ms	remaining: 4.03s
78:	learn: 2.2948702	total: 622m

243:	learn: 2.2726049	total: 1.9s	remaining: 2.69s
244:	learn: 2.2724912	total: 1.91s	remaining: 2.68s
245:	learn: 2.2724790	total: 1.92s	remaining: 2.68s
246:	learn: 2.2724378	total: 1.93s	remaining: 2.67s
247:	learn: 2.2723941	total: 1.93s	remaining: 2.66s
248:	learn: 2.2723649	total: 1.94s	remaining: 2.65s
249:	learn: 2.2723018	total: 1.95s	remaining: 2.64s
250:	learn: 2.2722818	total: 1.96s	remaining: 2.63s
251:	learn: 2.2722556	total: 1.96s	remaining: 2.63s
252:	learn: 2.2722306	total: 1.97s	remaining: 2.62s
253:	learn: 2.2721380	total: 1.98s	remaining: 2.61s
254:	learn: 2.2720764	total: 1.99s	remaining: 2.6s
255:	learn: 2.2719986	total: 1.99s	remaining: 2.59s
256:	learn: 2.2718417	total: 2s	remaining: 2.58s
257:	learn: 2.2718348	total: 2.01s	remaining: 2.58s
258:	learn: 2.2718291	total: 2.02s	remaining: 2.57s
259:	learn: 2.2717469	total: 2.02s	remaining: 2.56s
260:	learn: 2.2717375	total: 2.03s	remaining: 2.55s
261:	learn: 2.2716998	total: 2.04s	remaining: 2.54s
262:	learn: 2.271

423:	learn: 2.2661329	total: 3.3s	remaining: 1.28s
424:	learn: 2.2661035	total: 3.31s	remaining: 1.28s
425:	learn: 2.2660925	total: 3.32s	remaining: 1.27s
426:	learn: 2.2660854	total: 3.33s	remaining: 1.26s
427:	learn: 2.2660682	total: 3.34s	remaining: 1.25s
428:	learn: 2.2660676	total: 3.34s	remaining: 1.25s
429:	learn: 2.2660008	total: 3.35s	remaining: 1.24s
430:	learn: 2.2659846	total: 3.36s	remaining: 1.23s
431:	learn: 2.2659538	total: 3.37s	remaining: 1.22s
432:	learn: 2.2658984	total: 3.37s	remaining: 1.22s
433:	learn: 2.2658965	total: 3.38s	remaining: 1.21s
434:	learn: 2.2658783	total: 3.39s	remaining: 1.2s
435:	learn: 2.2658615	total: 3.4s	remaining: 1.19s
436:	learn: 2.2658075	total: 3.4s	remaining: 1.18s
437:	learn: 2.2657815	total: 3.41s	remaining: 1.18s
438:	learn: 2.2657190	total: 3.42s	remaining: 1.17s
439:	learn: 2.2657095	total: 3.43s	remaining: 1.16s
440:	learn: 2.2656726	total: 3.43s	remaining: 1.15s
441:	learn: 2.2656054	total: 3.44s	remaining: 1.14s
442:	learn: 2.26

====================== Start of Level 1 ======================
1 estimators included in Level 1 
====================== Start of Level 0 ======================
1 estimators included in Level 0 
====================== Start of Level 1 ======================
1 estimators included in Level 1 


In [75]:
df_trial = pd.DataFrame(mytrial)
df_trial[['nColumns','train_mae','remark']]

,nColumns,train_mae,remark
0,30,1.714346,NaN
1,30,1.714346,NaN
2,70,1.562609,try gbm selected columns
3,70,1.863692,try gbm selected columns
4,70,1.555698,try gbm selected columns
5,70,1.692624,try gbm selected columns


In [62]:
y_test_pred = df_trial.loc[2]['y_test_pred'].reshape(-1)
df_submit = pd.DataFrame()
df_submit['time_to_failure'] = y_test_pred
df_submit['seg_id'] = df_test['index']
df_submit.to_csv('submission_pystacknet_2.csv', index=False)